# Lab 2: Small-World Network (Refer to Lecture 7) 
Generate a regular network with N = 1000 nodes with degree 10. You may use either 
the Watts-Strogatz model [3] (by rewiring the edges randomly) or the Newman-
Watts model [4] (by adding random edges). 

I am going to use the Watts-Strogatz model in this Lab.

In [2]:
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import style
from functools import partial as part

style.use("seaborn")

wsg = nx.watts_strogatz_graph

## (a) Compute the average shortest distance L and average clustering coefficient C as a function of p, 
where p is the portion of rewired or added edges compared to the total number of initial edges. 

In [3]:
def LC4WSG(N=1000,K=10,p=0,seed=5054):
    g = wsg(N,K,p,seed)
    C = nx.average_clustering(g)
    L = nx.average_shortest_path_length(g)
    return L,C

In [4]:
LC4WSG(N=1000,K=10,p=.5)

(3.3849709709709708, 0.09651314796314847)

## (b) Make a plot of L and C against p, then compare your plot with Fig. 1 below. 
( You need to repeat (a) at least a few times for each p and take the average in 
order to get a smooth curve.) 

In [16]:
def AvgLC4WSG(N=1000,K=10,p=0,seed=5054,trails=1):
    Ls,Cs = zip(*map(part(LC4WSG,N,K,p),range(seed,seed+trails)))
    return (np.mean(Ls),np.std(Ls)),(np.mean(Cs),np.std(Cs))

In [15]:
ps=np.linspace(0,1,2)
Ls,Cs=zip(*map(part(AvgLC4WSG,1000,10),ps))
Lavg,Lstd=zip(*Ls)
Cavg,Cstd=zip(*Cs)

ValueError: too many values to unpack (expected 2)